# Alternative Infill Strategies for Expensive Multi-Objective Optimisation
[A. Rahat](http://emps.exeter.ac.uk/computer-science/staff/aamr201), [R. Everson](http://emps.exeter.ac.uk/computer-science/staff/reverson), and [J. Fieldsend](http://emps.exeter.ac.uk/computer-science/staff/jefields).   
[Department of Computer Science, University of Exeter, UK](http://emps.exeter.ac.uk/computer-science/).

>This repository contains Python code for the infill streategies presented in __Alternative Infill Strategies for Expensive Multi-Objective Optimisation__ by A. Rahat, R. Everson and J. Fieldsend, to appear in GECCO 2017 proceedings. Please refer to the _LICENSE_ before using the code. 

>Preprint repository: https://ore.exeter.ac.uk/repository/handle/10871/27157

>IPython notebook: http://nbviewer.jupyter.org/urls/bitbucket.org/arahat/gecco-2017/raw/6bafce8df19f40f1f0c92584b98efa063a59d594/ReadME.ipynb


## Pre-requisits.

The code here is a __Python3__ implementation of the infill strategies. The following modules are necessary to run the code here. 

* [DEAP](https://github.com/DEAP/deap)
* [Numpy](http://www.numpy.org/)
* [SciPy](https://www.scipy.org/)
* [matplotlib](https://matplotlib.org/2.0.0/index.html)
* [PyDOE](https://pythonhosted.org/pyDOE/)
* [evoalgos](https://ls11-www.cs.tu-dortmund.de/people/swessing/evoalgos/doc/)
* [GPy](https://github.com/SheffieldML/GPy)
* [CMA](https://www.lri.fr/~hansen/html-pythoncma/frames.html)

To install any of these modules, just issue the following command in your terminal. 

`$ pip install module_name`

To install a module from github, use appropriate command. For instance:

`$ pip install git+https://github.com/DEAP/deap`

In addition to these, we used a custom module written in _C_ for dominance comparison. The code is given in the repository. To install the module, use the following command wihtin the _FrontCalc_ directory.

`$ python setup.py install`

> __Note.__ As Python installations differ quite significantly, there may be other dependencies, but these should be standard modules from PyPi. If running the code results in complaints that something could not be imported, then please install the relevant module using _pip_.

## Setting up.

The multi-objective evolutionary optimiser method (_IscaOpt.Optimiser.EMO_) requires a _settings_ dictionary along with the multi-objective function and associated arguments or keyword arguments. We list the the most pertinent settings below. 

* n_dim (int): the number of dimensions in the parameter space.
* n_obj (int): the number of objectives, i.e. objective space dimensions.
* lb (list or numpy array): an array of lower bounds in parameter space (1 in each dimension).
* ub (list or numpy array): an array of upper bounds in parameter space (1 in each dimension).
* ref_vector (list or numpy array): reference vector in the objective space.
* method_name (str): the method to use for performing multi-objective optimisation (deafulats to 'HypI'). Options are:
    - 'HypI'
    - 'MSD'
    - 'DomRank'
    - 'MPoI'
    - 'SMSEGO'
    - 'ParEGO'
* budget (int): the budget on the number of function evaluations.
* n_samples (int): the number of initial samples.
* kern_name (str): the kernel function to be used with Gaussian Processes. Defaults to __'Matern52'__. Please refer to GPy documentation for other options. 
* s_vector (int): the number of scalarisation vectors for ParEGO.
* maxfevals (int): the maximum number of function evaluations for infill criterion optimisation using CMA-ES. Defaults to $20000d$, where $d$ is the number of dimensions in parameter space.  
* multisurrogate (bool): whether to use a multi-surrogate approach or not. Defaults to __False__, i.e. use a mono-surrogate approach. 
* cma_options (dict): dictionary of settings for Hansen's CMA-ES code. See [CMA-ES documentation](https://www.lri.fr/~hansen/html-pythoncma/frames.html) for more details on avaialble options.
* cma_sigma (float): the extent of the standard deviation for CMA-ES. See [CMA-ES documentation](https://www.lri.fr/~hansen/html-pythoncma/frames.html) for details. 
* init_file (str): intial design file. It should be a __.npz__ file with 'arr_0' set to decision variables matrix $X \in \mathbb{R}^{M \times n}$ and 'arr_1' for corresponding function response vector $\mathbf{f} \in \mathbb{R}^{M \times D}$ (please refer to the paper for details on notations).
* visualise (bool): it allows basic visualisations. Only available for the following cases: __n_obj=2__; __n_obj=1 and n_dim=2__; __n_obj=1 and n_dim=1__.

> __Notes__<br />
> * This package can be used for single objective Bayesian optimisation. To do so, specify the method by setting **method_name** to **'EGO'**, and of course **n_obj** to **1** with an appropriate function. <br />
> * For one-dimensional search space and a single objective problem, we just use a grid-search instead of CMA-ES. 




## Running the optimiser.

To run the optimiser it is sufficient to define an objective function that may produce a response given a decision vector, and use appropriate settings to call the multi-objective evolutionary optimiser method (_IscaOpt.Optimiser.EMO_). Here we give an example of using the optimiser. 

### An example.

_Problem description_: Use the optimiser to solve $2$-objective __DTLZ2__ problem, starting with $65$ initial LHS samples and a budget of $100$. 

In [ ]:
from IPython.display import clear_output
# example set up
import numpy as np
# import optimiser codes
import IscaOpt

settings = {\
    'n_dim': 6,\
    'n_obj': 2,\
    'lb': np.zeros(6),\
    'ub': np.ones(6),\
    'ref_vector': [2.5]*2,\
    'method_name': 'SMSEGO',\
    'budget':100,\
    'n_samples':65,\
    'visualise':True,\
    'multisurrogate':True}

# function settings
from deap import benchmarks as BM
fun = BM.dtlz2
args = (2,) # number of objectives as argument

# optimise
res = IscaOpt.Optimiser.EMO(fun, args, settings=settings)
clear_output()

Simulation settings. 
{'ub': array([ 1.,  1.,  1.,  1.,  1.,  1.]), 'budget': 100, 'n_obj': 2, 'method_name': 'SMSEGO', 'n_samples': 65, 'n_dim': 6, 'multisurrogate': True, 'ref_vector': [2.5, 2.5], 'lb': array([ 0.,  0.,  0.,  0.,  0.,  0.]), 'visualise': True}
<class 'IscaOpt.multi_surrogate.SMSEGO'> [<GPy.kern.src.stationary.Matern52 object at 0x7f6278075390>, <GPy.kern.src.stationary.Matern52 object at 0x7f623dfebfd0>]
method used:  SMSEGO
Episode:  1
[ 0.  0.  0.  0.  0.  0.] [ 1.  1.  1.  1.  1.  1.]
new X
Creating surrogate...
Optimiser name:  lbfgs
Creating surrogate...
Optimiser name:  lbfgs
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=398256, Mon Jul 17 20:12:43 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 5.186586561896789e-02 1.0e+00 2.15e-01  2e-01  2e-01 0:0.2
    2     18 -4.179764862367819e-03 1.1e+00 1.91e-01  2e-01  2e-01 0:0.2
    3     27 6.221142264656998e-02 1.2e+00 1.90e-01  2e-01  2e-01 0:0.3


 /usr/local/lib/python3.5/dist-packages/cma.py:957: FutureWarning:comparison to `None` will result in an elementwise object comparison in the future.


  100    900 -2.712371559278521e-01 2.9e+00 4.34e-05  3e-06  9e-06 0:3.6
  104    936 -2.712371559913764e-01 3.1e+00 3.38e-05  2e-06  6e-06 0:3.8
termination on tolfun=1e-07 (Mon Jul 17 20:12:45 2017)
final/bestever f-value = -2.712372e-01 -2.712372e-01
incumbent solution: [0.99999999999946265, 0.48864475110413541, 0.5021830059500263, 0.52968358079728883, 0.51312026853499981, 0.50554649984987055]
std deviation: [2.3310512073597672e-06, 6.2263089256131486e-06, 4.5161790138240513e-06, 5.6260796040835949e-06, 5.3121501468478791e-06, 4.8835934882292404e-06]
(9_w,18)-aCMA-ES (mu_w=5.4,w_1=30%) in dimension 6 (seed=398257, Mon Jul 17 20:12:45 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1    955 -1.707674500263323e-03 1.0e+00 2.95e-01  3e-01  3e-01 0:0.1
    2    973 1.563092358853830e-01 1.3e+00 3.26e-01  3e-01  3e-01 0:0.1
    3    991 -1.801279014401302e-03 1.4e+00 2.76e-01  2e-01  3e-01 0:0.2
   78   2341 -2.937549675999547e-01 4.4e+00 1.96e-04  4e-06

 /usr/local/lib/python3.5/dist-packages/deap/creator.py:141: RuntimeWarning:A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
 /usr/local/lib/python3.5/dist-packages/deap/creator.py:141: RuntimeWarning:A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.


   95    855 -2.615026886539207e-01 3.2e+00 5.02e-05  3e-06  9e-06 0:3.4
termination on tolfun=1e-07 (Mon Jul 17 20:16:21 2017)
final/bestever f-value = -2.615027e-01 -2.615027e-01
incumbent solution: [0.9999999999798509, 0.48967844336359201, 0.50344335434851906, 0.52015473051480399, 0.51391714465569049, 0.49781146750384847]
std deviation: [3.0843122376069839e-06, 8.7104022409196848e-06, 9.1180490019514933e-06, 8.9424035942750674e-06, 7.8461263565720552e-06, 7.1396868535229913e-06]
(9_w,18)-aCMA-ES (mu_w=5.4,w_1=30%) in dimension 6 (seed=382136, Mon Jul 17 20:16:21 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1    874 3.876293058774527e-02 1.0e+00 2.82e-01  3e-01  3e-01 0:0.1
    2    892 2.818299065687135e-02 1.4e+00 3.32e-01  3e-01  4e-01 0:0.1
    3    910 -2.171363844132124e-02 1.7e+00 3.41e-01  3e-01  4e-01 0:0.2
   83   2350 -2.615026887987817e-01 3.0e+00 1.59e-04  3e-06  9e-06 0:5.0
termination on tolfun=1e-07 after 1 restart (Mon Jul 17 20:1

In the Figure above, the blue crosses show the initial samples, and the solid circle show the newly sampled solutions with darker colours showing earlier samples. The black encircled solid is the latest sample.

## Errata

* Equation (5) should be as follows. 
    $\alpha(\mathbf{x}, f^*) = \int_{-\infty}^{\infty}I(\mathbf{x}, f^*)P(\hat{f}|\mathbf{x},\mathcal{D})d\hat{f} = \sigma(\mathbf{x})(s\Phi(s) + \phi(s))$.

## Contact

For any comments, queries or suggestions, please send an email to: __a.a.m.rahat@exeter.ac.uk__